In [1]:
""" Import dependencies """
import os
import keras
import numpy as np
from sklearn import metrics
from ML.ml_utils import ML_utils, CustomLoss
import matplotlib
import tensorflow
from matplotlib import pyplot as plt

%matplotlib inline

Segmentation Models: using `keras` framework.


2022-04-28 13:22:13.637789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:22:13.672335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:22:13.672624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
""" User defined parameters """
matplotlib.use('agg')
plt.ioff()
dataset_name = 'data_medium'
path_to_model = '/localhome/studenter/renatask/Project/ML/models/Deeplab_mini'
model_name = os.path.split(path_to_model)[1] + '/'
save_path_image_predictions = '/localhome/studenter/renatask/Project/ML/predicted_images/'

In [3]:
""" Load model """
ml = ML_utils(user='renatask')
model = keras.models.load_model(path_to_model, custom_objects={'call':CustomLoss.call})
4
""" Define test image paths """
path_to_images = "data/datasets/" + dataset_name + "/test/images"
path_to_masks = "data/datasets/" + dataset_name + "/test/masks"

""" Define classes """
classes = {
    0: 'not_water',
    1: 'water'
}

2022-04-28 13:22:21.160114: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-28 13:22:21.161885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:22:21.162180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:22:21.162396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [5]:
""" Produce images """
n_images = len([img for img in os.listdir(path_to_images) if not img.startswith('.')])
test_data_generator = ml.DataGenerator(path_to_images,path_to_masks,train=False)
n_classes = len(classes)
predict_batch = 32
n_batch = n_images // 32
if not n_images % 32 == 0: n_batch += 1
images = []
predictions = []
segmentations = []

""" Predict images """
for batch in range(0,n_batch):
    imgs, masks = next(test_data_generator)
    for i in range(0, len(imgs)):
        img = np.expand_dims(imgs[i],axis=0)
        images.append(imgs[i])
        predictions.append(model.predict(img))
        segmentations.append(masks[i])

2022-04-27 18:15:52.171310: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2022-04-27 18:15:52.924958: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-04-27 18:15:52.944756: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-04-27 18:15:52.944788: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-04-27 18:15:52.944899: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

In [6]:
""" Set predictions to maximum value (single-band) """
for i in range(len(predictions)):
    predictions[i] = np.argmax(predictions[i],axis=-1)
    segmentations[i] = np.argmax(segmentations[i],axis=-1)

In [7]:
""" Save predicted images for visualization """
if not os.path.exists(save_path_image_predictions + model_name):
    os.makedirs(save_path_image_predictions + model_name)

count = 0
for img, pred, mask in zip(images,predictions,segmentations):
    pred = np.rollaxis(pred,0,3)
    fig, axs = plt.subplots(1,3,figsize=(25,25))
    plt.tight_layout()

    axs[0].imshow(img[:,:,0],cmap='PuBuGn_r')
    axs[1].imshow(pred)
    axs[2].imshow(mask)

    axs[0].set_title("Original Image")
    axs[1].set_title("Prediction")
    axs[2].set_title("Ground Truth")

    plt.savefig(save_path_image_predictions + model_name + str(count) + '.png')
    count += 1

/tmp/ipykernel_2453998/2856699180.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(1,3,figsize=(25,25))


In [8]:
""" Calculate performance """
# Flatten arrays
print("[INFO] Flattening all numpy arrays... this may take a couple of minutes.")
predictions = np.concatenate([np_array.ravel() for np_array in predictions])
segmentations = np.concatenate([np_array.ravel() for np_array in segmentations])

print("[INFO] Calculating precision/recall...")
precision = metrics.precision_score(segmentations, predictions, average='weighted')
recall = metrics.recall_score(segmentations, predictions, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {(2*precision*recall)/(recall+precision)}")
print(f"Confusion matrix: \n {metrics.confusion_matrix(segmentations, predictions)}")

[INFO] Flattening all numpy arrays... this may take a couple of minutes.
[INFO] Calculating precision/recall...
Precision: 0.9843223022569684
Recall: 0.9845417952585425
F1-Score: 0.9844320365229908
Confusion matrix: 
 [[ 28454219   5519086]
 [  1022015 388152200]]


In [14]:
""" Remove this cell """
import rasterio
from skimage import exposure

image_path = '/localhome/studenter/renatask/Project/data/processed_downloads/gaula_melhus_S1A_IW_GRDH_1SDV_20190907T165510.tif'
img_name = 'gaula_melhus_S1A_IW_GRDH_1SDV_20190907T16551'
mask_path = '/localhome/studenter/renatask/Project/data/processed_masks/gaula_melhus.tif'

image, mask = rasterio.open(image_path).read(), rasterio.open(mask_path).read()
image = np.rollaxis(image,0,3)
mask = np.rollaxis(mask,0,3)
bands = image.shape[-1]

rescaled_image = np.zeros(image.shape,dtype="float32")
for b in range(bands):
    p2 = np.percentile(image[:,:,b],2)
    p98 = np.percentile(image[:,:,b],98)
    rescaled_band = exposure.rescale_intensity(image[:,:,b],in_range=(p2,p98),out_range=(0,1))
    rescaled_image[:,:,b] = rescaled_band
image = rescaled_image


mask_binned = ml.bin_image(mask)
mask = ml.getSegmentationArr(mask_binned, ml.N_CLASSES)

H_32 = int(image.shape[0]/32)*32
W_32 = int(image.shape[1]/32)*32

image = image[0:H_32,0: W_32,:]
mask = mask[0:H_32, 0:W_32,:]

image = np.expand_dims(image,axis=0)

prediction = model.predict(image)

image = image.squeeze()
prediction = np.argmax(prediction,axis=-1)
mask = np.argmax(mask,axis=-1)

prediction = np.rollaxis(prediction,0,3)
fig, axs = plt.subplots(1,3,figsize=(25,25))
plt.tight_layout()

axs[0].imshow(image[:,:,0],cmap='PuBuGn_r')
axs[1].imshow(prediction)
axs[2].imshow(mask)

axs[0].set_title("Original Image")
axs[1].set_title("Prediction")
axs[2].set_title("Ground Truth")

plt.savefig('/localhome/studenter/renatask/Project/ML/predicted_images/' + model_name + 'random/' +img_name+ '_' + str(count) + '.png')
count += 1





In [4]:
""" Remove this cell """
import rasterio
from skimage import exposure

image_path = '/localhome/studenter/renatask/Project/data/datasets/data_mini/val/images/1.tif'
img_name = '1'


image = rasterio.open(image_path).read()
image = np.rollaxis(image,0,3)
bands = image.shape[-1]

rescaled_image = np.zeros(image.shape,dtype="float32")
for b in range(bands):
    p2 = np.percentile(image[:,:,b],2)
    p98 = np.percentile(image[:,:,b],98)
    rescaled_band = exposure.rescale_intensity(image[:,:,b],in_range=(p2,p98),out_range=(0,1))
    rescaled_image[:,:,b] = rescaled_band
image = rescaled_image


H_32 = int(image.shape[0]/32)*32
W_32 = int(image.shape[1]/32)*32

image = image[0:H_32,0: W_32,:]

image = np.expand_dims(image,axis=0)

prediction = model.predict(image)

image = image.squeeze()
prediction = np.argmax(prediction,axis=-1)

prediction = np.rollaxis(prediction,0,3)
fig, axs = plt.subplots(1,2,figsize=(25,25))
plt.tight_layout()

axs[0].imshow(image[:,:,0],cmap='PuBuGn_r')
axs[1].imshow(prediction)

axs[0].set_title("Original Image")
axs[1].set_title("Prediction")

plt.savefig('/localhome/studenter/renatask/Project/ML/predicted_images/' + model_name + 'test/' +img_name+ '_' + str(count) + '.png')
count += 1


NameError: name 'model' is not defined